In [2]:
import numpy as np
from scipy.optimize import minimize

In [46]:
n = 10
a = np.array([np.random.uniform(3, 7) for _ in range(n)])

t_i = lambda f_i, c_i, a_i: a_i + (f_i/c_i)**3
T = lambda f, c, a=a: np.sum(t_i(fi, ci, ai)*fi for fi, ci, ai in zip(f, c, a))

In [47]:
def get_k_best(l, c, k=5):
    """
    Finds k best rows in l
    
    takes:
        l -- matrix of T_i, f, c
        k -- k best rows
    returns:
        k best rows
    """
    mat = l
    sor = c[np.argsort(mat)]
    return sor[:k]

In [68]:
def cross_breed(vects, constrain, k=5):
    """
    Computes cross breeing of vectors
    
    takes:
        vects      -- vectors, that new generation would be created from
        constrain  -- (float) size of new vector space (kinda)
        k          -- amount vectors that would be used to produce new child
        
    returns:
        new vector
    """
    coeff = np.random.dirichlet(np.ones(vects.shape[0]), size=1).reshape(-1)
    
    conv_mat = np.array([ci * vi for ci, vi in zip(coeff, vects)])
    new_vec = conv_mat.sum(axis=0)
    return new_vec

In [69]:
def gen_population(dim_pop, dim_c=n, C=150, costs=None, Ts = None):
    """
    Generates population. 
    On first step, when no c initialized, generates random c. 
    On next steps calculates c using cross of gens.
    
    takes: 
        dim_pop -- number of populations (how many rows in returned matrix)
        dim_c   -- dimention of population (how many columns in returned matrix)
        C       -- max cost sum (sum(c_i) <= C)
        costs   -- passed if it is desired to calculate not on first step
    returns:
        matrix c[dim_pop, dim_c]
    """
    c = np.zeros((dim_pop, dim_c))
    if costs is None:
        for i in range(dim_pop):
            c[i] = np.array([np.random.uniform(0, C/dim_c) for j in range(dim_c)])
    else:
        for i in range(dim_pop):
            # number of used vects
            k = 5
            k_best = get_k_best(Ts, costs, k)
            c[i] = cross_breed(k_best, 2, k)
    return c

In [70]:
def mineze(c, F = 500, m=n):
    """
    Calculates step of inner minimization. 
    
    takes:
        c -- vector of costs
        F -- total flow of network
        m -- number of connections in graph
        
    returns:
        tuple: T_i, f, c
        T_i -- value in min point f
        f   -- found min with specified c
        c   -- passed c
    """
    cons = ({'type' : 'eq',  'fun' : lambda x: np.sum(x) - F}, 
            {'type' : 'ineq','fun' : lambda x: x            })
    f = np.ones(n) * F/m

    f = minimize(lambda x: T(x, c), f, constraints=cons).x
    T_i = T(f, c)
    return T_i

In [81]:
from sklearn.metrics import euclidean_distances

c = None
Tfc = None
T_prev = 0
prec = 1

epoch = 0
while prec > 1e-3:
    c = gen_population(10, costs=c, Ts=Tfc)
    Tfc = []
    for c_i in c:
        Tfc.append(mineze(c_i))

    prec = euclidean_distances(sorted(Tfc)[0], T_prev)
    T_prev = sorted(Tfc)[0]
    epoch = epoch + 1
    
    print('epoch: {}'.format(epoch))
    print('T value: {}'.format(Tfc[0]))
    print('costs: {}'.format(c[0]))
    print('precision: {}\n'.format(prec))

epoch: 1
T value: 123684.01997637126
costs: [  8.4686581    8.48840878  14.98972515   9.27022912   9.7036351
   1.23490906   7.60818132   9.65852408   9.25078609   1.60281813]
precision: [[ 74062.1486633]]

epoch: 2
T value: 140798.17343485795
costs: [  9.12340584   8.93423586  11.54037105  12.6007003    9.12921369
   4.40889107   5.60884676   5.36398543   6.68056179   3.40721482]
precision: [[ 33043.01883841]]

epoch: 3
T value: 129180.99406120712
costs: [  8.84971358   8.17571712  12.03146277  10.75244105   7.69352132
   4.18714491   6.89706061   7.4100921    8.08502145   4.99602   ]
precision: [[ 7940.24251713]]

epoch: 4
T value: 121543.35402860689
costs: [  8.81999368   9.23640823  11.89674256  11.40589544   7.52457288
   4.94260391   7.57776533   6.55298439   7.95253148   4.8296258 ]
precision: [[ 2993.79890941]]

epoch: 5
T value: 120763.3209326844
costs: [  8.90763996   9.11116337  11.84264173  11.28060129   7.39241397
   4.99232852   7.67329059   6.5943399    8.04399412   5.07

In [75]:
cons = ({'type' : 'ineq',  'fun' : lambda x: -np.sum(x) + 150}, 
        {'type' : 'ineq','fun' : lambda x: x            })

In [86]:
f = np.ones(10) * 500/10
minimize(lambda x: T(c=x, f=f), gen_population(1), constraints=cons).x

array([  4.92537014,  13.1466713 ,   1.61835038,   2.65746521,
         3.91618473,  11.56997657,   0.37409701,  13.28911962,
         9.66617592,  13.02562941])

In [17]:
print(np.random.dirichlet(np.ones(3),size=1))

[[ 0.14734939  0.79738904  0.05526157]]


In [20]:
np.zeros(3).shape[0]

3

In [83]:
c[0]

array([  9.05067241,   9.02930228,  11.87588046,  11.18277378,
         7.42027638,   4.94693727,   7.77108801,   6.5768324 ,
         8.1257274 ,   5.13172594])